# Running cellpose in colab with a GPU

(thanks to Matteo Carandini for setting this up)

UPDATED DEC 2020 for TORCH VERSION cellpose v0.6

UPDATED NOV 2021 for cellpose / omnipose v0.7

## installation

Install cellpose -- by default the torch GPU version is installed in COLAB notebook.

Note that cellpose uses the latest version of numpy, so please click the "Restart runtime" button once the install completes.

Check CUDA version and GPU

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_22:08:44_Pacific_Standard_Time_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Sun Mar  6 11:48:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000    WDDM  | 00000000:73:00.0  On |                  Off |
| 33%   30C    P8    21W / 260W |    607MiB / 49152MiB |      8%      Default |
|                    

import libraries and check GPU (the first time you import cellpose the models will download).

In [2]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import tifffile as tf
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import zarr
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
from cellpose import models, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup();


>>> GPU activated? 1
2022-03-06 11:49:07,919 [INFO] WRITING LOG OUTPUT TO C:\Users\Linux\.cellpose\run.log


Load your images

In [4]:
# REPLACE FILES WITH YOUR IMAGE PATHS
files = [r"D:\EHG\Explants\uncomp\output\C2-eh3931_cyclo25_explant-1_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-eh3931_cyclo25_explant-2_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-eh3931_cyclo25_explant-3_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-eh3931_cyclo25_explant-4_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-EH3931_DMSOEtOH_Explant-1_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-EH3931_DMSOEtOH_Explant-2_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-EH3931_DMSOEtOH_Explant-3_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-EH3931_SAG7_Explant-1_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-EH3931_SAG7_Explant-2_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_stitched_C0.ome_Processed.aivia_Processed.aivia.tif",
r"D:\EHG\Explants\uncomp\output\C2-EH3931_SAG7_Explant-3_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif"]
    
imgs = [tf.imread(f) for f in files]
nimg = len(imgs)

## run cellpose 3D

There are two ways to run cellpose in 3D, this cell shows both, choose which one works best for you.

First way: computes flows from 2D slices and combines into 3D flows to create masks



In [ ]:
# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')


# test 3D stack

img_3D = imgs

# * with 3D you have to set the diameter manually (no auto detect) *

### TWO WAYS TO RUN CELLPOSE IN 3D

# 1. computes flows from 2D slices and combines into 3D flows to create masks
print('running cellpose 2D slice flows => masks')
masks, flows, styles, diams = model.eval(imgs, channels=[0,0], diameter=15, do_3D=True)


Second way: computes masks in 2D slices and stitches masks in 3D based on mask overlap

Note stitching (with stitch_threshold > 0) can also be used to track cells over time.

In [5]:
# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')

# 2. computes masks in 2D slices and stitches masks in 3D based on mask overlap
print('running cellpose 2D + stitching masks')
masks_stitched, flows_stitched, styles_stitched, _ = model.eval(imgs, channels=[0,0], diameter=15, do_3D=False, stitch_threshold=0.5)

2022-03-06 11:53:22,254 [INFO] ** TORCH CUDA version installed and working. **
2022-03-06 11:53:22,254 [INFO] >>>> using GPU
running cellpose 2D + stitching masks
2022-03-06 11:53:22,379 [INFO] ~~~ FINDING MASKS ~~~
2022-03-06 11:53:22,381 [INFO] 0%|          | 0/10 [00:00<?, ?it/s]
2022-03-06 11:53:22,381 [INFO] multi-stack tiff read in as having 267 planes 1 channels
2022-03-06 11:53:30,560 [INFO] 
2022-03-06 11:53:30,560 [INFO] 0%|          | 0/267 [00:00<?, ?it/s]
2022-03-06 11:53:30,560 [INFO] 
2022-03-06 11:53:53,574 [INFO] 
2022-03-06 11:53:53,574 [INFO] 0%|          | 1/267 [00:23<1:42:01, 23.01s/it]
2022-03-06 11:53:53,574 [INFO] 
2022-03-06 11:54:12,966 [INFO] 
2022-03-06 11:54:12,966 [INFO] 1%|          | 2/267 [00:42<1:32:14, 20.88s/it]
2022-03-06 11:54:12,966 [INFO] 
2022-03-06 11:54:32,427 [INFO] 
2022-03-06 11:54:32,427 [INFO] 1%|1         | 3/267 [01:01<1:29:01, 20.23s/it]
2022-03-06 11:54:32,427 [INFO] 
2022-03-06 11:54:51,821 [INFO] 
2022-03-06 11:54:51,821 [INFO] 1%|

### save output to *_seg.npy

you will see the files save in the Files tab and you can download them from there

In [6]:
from cellpose import io
io.save_masks(imgs, masks_stitched, flows_stitched, files, tif=True, save_flows=True)

In [ ]:
io.masks_flows_to_seg(imgs, masks_stitched, flows_stitched, diams, files)

Results from 3D flows => masks computation

In [ ]:
# DISPLAY RESULTS
from cellpose import plot

# DISPLAY RESULTS 3D flows => masks
plt.figure(figsize=(15,3))
for i,iplane in enumerate(np.arange(0,75,10,int)):
  img0 = plot.image_to_rgb(img_3D[iplane, [1,0]].copy(), channels=[2,3])
  plt.subplot(1,8,i+1)
  outlines = utils.masks_to_outlines(masks[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)

Results from stitching

In [ ]:
# DISPLAY RESULTS stitching
plt.figure(figsize=(15,3))
for i,iplane in enumerate(np.arange(0,75,10,int)):
  img0 = plot.image_to_rgb(img_3D[iplane, [1,0]].copy(), channels=[2,3])
  plt.subplot(1,8,i+1)
  outlines = utils.masks_to_outlines(masks_stitched[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)